# Install Library

In [ ]:
# @title Create a prompt

import google.generativeai as genai
from google.colab import userdata

api_key_name = 'GOOGLE_API_KEY' # @param {type: "string"}
prompt = 'What is the velocity of an unladen swallow?' # @param {type: "string"}
system_instructions = 'You have a tendency to speak in riddles.' # @param {type: "string"}
model = 'gemini-2.0-flash' # @param {type: "string"} ["gemini-1.0-pro", "gemini-1.5-pro", "gemini-1.5-flash", "gemini-2.0-flash"]
temperature = 0.5 # @param {type: "slider", min: 0, max: 2, step: 0.05}
stop_sequence = '' # @param {type: "string"}

if model == 'gemini-1.0-pro' and system_instructions is not None:
  system_instructions = None
  print('\x1b[31m(WARNING: System instructions ignored, gemini-1.0-pro does not support system instructions)\x1b[0m')

if model == 'gemini-1.0-pro' and temperature > 1:
  temperature = 1
  print('\x1b[34m(INFO: Temperature set to 1, gemini-1.0-pro does not support temperature > 1)\x1b[0m')

if system_instructions == '':
  system_instructions = None

api_key = userdata.get(api_key_name)
genai.configure(api_key=api_key)
model = genai.GenerativeModel(model, system_instruction=system_instructions)
config = genai.GenerationConfig(temperature=temperature, stop_sequences=[stop_sequence])
response = model.generate_content(contents=[prompt], generation_config=config)
response.text

In [ ]:
pip install xlsxwriter==3.1.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 7.4 MB/s eta 0:00:00


In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'masterdata-farmu' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [ ]:
import numpy as np
import pandas as pd

# Import Data

### df_base

In [ ]:
from google.cloud import bigquery
from google.cloud import storage, bigquery
from google.colab import auth
from google.colab import data_table
import pandas as pd
import numpy as np

def get_bigquery_data(client,query: str) -> pd.DataFrame:
    df = client.query(query).result().to_dataframe()
    return df

if __name__ == "__main__":
    project = 'masterdata-farmu'
    location = 'US'
    client_ = bigquery.Client(project=project, location=location)
    data_table.enable_dataframe_formatter()
    auth.authenticate_user()

    new_query = '''
            WITH DATA_REPORT AS
            (
              SELECT
                co.sku AS COD_PROD,
                UPPER(dm.product) AS DESCRIPCION_PROD,
                dm.maker AS LABORATORIO,
                CAST(dm.ean AS STRING) AS COD_BARRA,
                CAST(customer.customer_id AS STRING) AS COD_CLI,
                customer.name AS NOMBRE_CLI,
                co.quantity AS UNIDADES,
                co.unit_price AS PRECIO_UNITARIO,
                co.order_date AS FECHA,
                UPPER(co.sales_channel) AS CANAL_VENTA,
                UPPER(customer.address) AS DIRECCION,
                UPPER(customer.city) AS CIUDAD,
                CASE
                          WHEN (customer.department) IN ('BOGOTÁ, D.C.', 'BOGOTÁ', 'BOGOTA', 'BOGOTA / PUENTE ARANDA') THEN 'BOGOTÁ, D.C.'
                          WHEN (customer.department) IN ('BARRANQUILLA', 'BARRAQUILLA') THEN 'ATLÁNTICO'
                          WHEN (customer.department) IN ('SOACHA', 'CHOCONTÁ') THEN 'CUNDINAMARCA'
                          ELSE UPPER(customer.department)
                END AS DEPARTAMENTO
              FROM `masterdata-farmu.db_source_of_truth.revenue`  AS co
              LEFT JOIN `masterdata-farmu.db_Sales.dim_catalog` AS dm
              ON co.sku = dm.sku
              LEFT JOIN `masterdata-farmu.db_source_of_truth.customers` AS customer
              ON co.drugstore_id=customer.drugstore_id
              WHERE 1=1
              AND cancelated_at IS NULL
              AND marketplace IS false
            )

            SELECT *
            FROM DATA_REPORT
            WHERE 1=1
                  AND FECHA >= DATE_SUB(DATE_SUB(CURRENT_DATE(), INTERVAL EXTRACT(DAY FROM CURRENT_DATE())-1 DAY), INTERVAL 1 month)
                  AND FECHA < DATE_SUB(DATE_SUB(CURRENT_DATE(), INTERVAL EXTRACT(DAY FROM CURRENT_DATE())-1 DAY), INTERVAL 0 month)
                  AND DESCRIPCION_PROD NOT LIKE '%SIN INFO%'
                  AND NOMBRE_CLI IS NOT NULL
                  AND NOMBRE_CLI <> ''
                  AND NOMBRE_CLI NOT LIKE '%test%'
                  AND PRECIO_UNITARIO > 0
                  AND COD_BARRA IS NOT NULL
                  AND COD_BARRA NOT IN ('','Falta Encontrarlo')
                  AND DIRECCION IS NOT NULL
                  AND COD_CLI IS NOT NULL
                  AND COD_CLI != '0'
                  AND UNIDADES >= 1
              ORDER BY FECHA ASC

            '''
    df_base = get_bigquery_data(client_,new_query)

In [ ]:
from google.cloud import bigquery
from google.cloud import storage, bigquery
from google.colab import auth
from google.colab import data_table
import pandas as pd
import numpy as np

def get_bigquery_data(client,query: str) -> pd.DataFrame:
    df = client.query(query).result().to_dataframe()
    return df

if __name__ == "__main__":
    project = 'masterdata-farmu'
    location = 'US'
    client_ = bigquery.Client(project=project, location=location)
    data_table.enable_dataframe_formatter()
    auth.authenticate_user()

    new_query_marketplace = '''
    WITH DATA_REPORT AS
                (
                  SELECT
                    co.ean AS COD_PROD,
                    UPPER(co.description) AS DESCRIPCION_PROD,
                    dm.maker AS LABORATORIO,
                    CAST(dm.ean AS STRING) AS COD_BARRA,
                    CAST(customer.customer_id AS STRING) AS COD_CLI,
                    customer.name AS NOMBRE_CLI,
                    co.quantity AS UNIDADES,
                    co.price AS PRECIO_UNITARIO,
                    DATE(co.creation_date) AS FECHA,
                    'Tradicional' AS CANAL_VENTA,
                    UPPER(customer.address) AS DIRECCION,
                    UPPER(customer.city) AS CIUDAD,
                    CASE
                              WHEN (customer.department) IN ('BOGOTÁ, D.C.', 'BOGOTÁ', 'BOGOTA', 'BOGOTA / PUENTE ARANDA') THEN 'BOGOTÁ, D.C.'
                              WHEN (customer.department) IN ('BARRANQUILLA', 'BARRAQUILLA') THEN 'ATLÁNTICO'
                              WHEN (customer.department) IN ('SOACHA', 'CHOCONTÁ') THEN 'CUNDINAMARCA'
                              ELSE UPPER(customer.department)
                    END AS DEPARTAMENTO
                  FROM `masterdata-marketplace.db_source_of_truth.revenue`  AS co
                  LEFT JOIN `masterdata-farmu.db_Sales.dim_catalog` AS dm
                    ON co.ean = dm.ean
                  LEFT JOIN `masterdata-farmu.db_source_of_truth.customers` AS customer
                  ON co.customer_email=customer.drugstore_email
                  WHERE 1=1
                  -- AND cancelated_at IS NULL
                  AND status != 'cancelled'
                )

                SELECT *
                FROM DATA_REPORT
                WHERE 1=1
                      AND FECHA >= DATE_SUB(DATE_SUB(CURRENT_DATE(), INTERVAL EXTRACT(DAY FROM CURRENT_DATE())-1 DAY), INTERVAL 1 month)
                      AND FECHA < DATE_SUB(DATE_SUB(CURRENT_DATE(), INTERVAL EXTRACT(DAY FROM CURRENT_DATE())-1 DAY), INTERVAL 0 month)
                      AND DESCRIPCION_PROD NOT LIKE '%SIN INFO%'
                      AND NOMBRE_CLI IS NOT NULL
                      AND NOMBRE_CLI <> ''
                      AND NOMBRE_CLI NOT LIKE '%test%'
                      AND PRECIO_UNITARIO > 0
                      AND COD_BARRA IS NOT NULL
                      AND COD_BARRA NOT IN ('','Falta Encontrarlo')
                      AND DIRECCION IS NOT NULL
                      AND COD_CLI IS NOT NULL
                      AND COD_CLI != '0'
                      AND UNIDADES >= 1
                  ORDER BY FECHA ASC


    '''

    df_base = get_bigquery_data(client_,new_query_marketplace)

# CLIENTES

In [ ]:
CLIENTES_iqvia = df_base[['COD_CLI', 'NOMBRE_CLI', 'DIRECCION', 'CIUDAD', 'DEPARTAMENTO']].copy()

CLIENTES_iqvia = CLIENTES_iqvia.sort_values('COD_CLI')

CLIENTES_iqvia.head()

,COD_CLI,NOMBRE_CLI,DIRECCION,CIUDAD,DEPARTAMENTO
19003,5889192460475,Droguería Superfarma Th,"CRA. 80 #72-36, BOGOTÁ, COLOMBIA",BOGOTÁ,BOGOTÁ
35707,5889207632059,Su Farmacia Amiga,"CRA. 74 #51A-13, BOGOTÁ, COLOMBIA",BOGOTÁ,BOGOTÁ
23654,5889207632059,Su Farmacia Amiga,"CRA. 74 #51A-13, BOGOTÁ, COLOMBIA",BOGOTÁ,BOGOTÁ
23655,5889207632059,Su Farmacia Amiga,"CRA. 74 #51A-13, BOGOTÁ, COLOMBIA",BOGOTÁ,BOGOTÁ
43203,5889207632059,Su Farmacia Amiga,"CRA. 74 #51A-13, BOGOTÁ, COLOMBIA",BOGOTÁ,BOGOTÁ


In [ ]:
CLIENTES_closeup = df_base[['COD_CLI', 'NOMBRE_CLI', 'DIRECCION', 'CIUDAD', 'DEPARTAMENTO']].copy()

CLIENTES_closeup = CLIENTES_closeup.rename(columns={'COD_CLI': 'CODIGO_CLIENTE', 'NOMBRE_CLI': 'RAZON_SOCIAL'})

CLIENTES_closeup = CLIENTES_closeup.sort_values('CODIGO_CLIENTE')

CLIENTES_closeup.head()

,CODIGO_CLIENTE,RAZON_SOCIAL,DIRECCION,CIUDAD,DEPARTAMENTO
19003,5889192460475,Droguería Superfarma Th,"CRA. 80 #72-36, BOGOTÁ, COLOMBIA",BOGOTÁ,BOGOTÁ
35707,5889207632059,Su Farmacia Amiga,"CRA. 74 #51A-13, BOGOTÁ, COLOMBIA",BOGOTÁ,BOGOTÁ
23654,5889207632059,Su Farmacia Amiga,"CRA. 74 #51A-13, BOGOTÁ, COLOMBIA",BOGOTÁ,BOGOTÁ
23655,5889207632059,Su Farmacia Amiga,"CRA. 74 #51A-13, BOGOTÁ, COLOMBIA",BOGOTÁ,BOGOTÁ
43203,5889207632059,Su Farmacia Amiga,"CRA. 74 #51A-13, BOGOTÁ, COLOMBIA",BOGOTÁ,BOGOTÁ


# FACMES

In [ ]:
FACMES_iqvia = df_base[['COD_PROD', 'DESCRIPCION_PROD', 'COD_CLI', 'NOMBRE_CLI', 'UNIDADES', 'PRECIO_UNITARIO', 'FECHA', 'CANAL_VENTA']].copy()

FACMES_iqvia = FACMES_iqvia.sort_values('FECHA')

FACMES_iqvia.head()

,COD_PROD,DESCRIPCION_PROD,COD_CLI,NOMBRE_CLI,UNIDADES,PRECIO_UNITARIO,FECHA,CANAL_VENTA
0,7703038050179,CIPROFLOXACINA 500MG CAJA X300 TABLETAS LAPROFF,6030340948155,Drogueria Santafe De Fatima,1,101887.500000000,2025-07-01,Tradicional
1194,7702184590096,FARMALAX LAXANTE BISACODILO 5MG CAJA X100 GRAG...,6175487492283,La Nueva Vida,1,12315.600000000,2025-07-01,Tradicional
1193,7706263200630,PAX NOCHE LIMÓN CAJA X24 SOBRES,6175487492283,La Nueva Vida,1,49500.150000000,2025-07-01,Tradicional
1192,7706263200616,PAX DÍA NARANJA ACETAMINOFÉN + FENILEFRINA CAJ...,6175487492283,La Nueva Vida,1,50899.800000000,2025-07-01,Tradicional
1191,7703546030601,CONGESTEX CAJA X60 CÁPSULAS,6175487492283,La Nueva Vida,1,47998.650000000,2025-07-01,Tradicional


In [ ]:
FACMES_closeup = df_base[['COD_CLI', 'COD_PROD', 'UNIDADES', 'PRECIO_UNITARIO', 'FECHA']].copy()

FACMES_closeup = FACMES_closeup.rename(columns={'COD_CLI': 'CODIGO_CLIENTE', 'COD_PROD': 'CODIGO_PRODUCTO'})

FACMES_closeup = FACMES_closeup.sort_values('FECHA', ascending=False)

FACMES_closeup.head()

,CODIGO_CLIENTE,CODIGO_PRODUCTO,UNIDADES,PRECIO_UNITARIO,FECHA
48805,6108334260411,7706569001191,4,6050.000000000,2025-07-31
47442,7046216614075,7703038061038,9,3083.300000000,2025-07-31
47412,7231971590331,7707019384116,2,8400.000000000,2025-07-31
47413,7231971590331,7703432998916,6,2989.800000000,2025-07-31
47414,7231971590331,7707321982093,5,7580.100000000,2025-07-31


# PRODUCTOS

In [ ]:
PRODUCTOS_iqvia = df_base[['COD_PROD', 'DESCRIPCION_PROD', 'LABORATORIO', 'PRECIO_UNITARIO', 'COD_BARRA']].copy()

PRODUCTOS_iqvia = PRODUCTOS_iqvia.sort_values('DESCRIPCION_PROD')

PRODUCTOS_iqvia.head()

,COD_PROD,DESCRIPCION_PROD,LABORATORIO,PRECIO_UNITARIO,COD_BARRA
19147,7709521995521,01NE DAY NARANJA X30 SOB PAG24 LLV30,NEW HEALTH,21143.200000000,7709521995521
33148,7709521995521,01NE DAY NARANJA X30 SOB PAG24 LLV30,NEW HEALTH,21143.200000000,7709521995521
46703,4104480706172,ABRILAR HEDERA HELIX 7MG/ML JARABE FRASCO X200...,MEGALABS,48899.550000000,4104480706172
48760,4104480706172,ABRILAR HEDERA HELIX 7MG/ML JARABE FRASCO X200...,MEGALABS,48899.550000000,4104480706172
825,4104480705151,ABRILAR HEDERA HELIX JARABE X100 ML,MEGALABS,34899.900000000,4104480705151


In [ ]:
PRODUCTOS_closeup = df_base[['COD_PROD', 'COD_BARRA', 'DESCRIPCION_PROD', 'LABORATORIO', 'PRECIO_UNITARIO']].copy()

PRODUCTOS_closeup = PRODUCTOS_closeup.rename(columns={'COD_PROD': 'CODIGO_PRODUCTO',
                                                      'COD_BARRA': 'EAN',
                                                      'DESCRIPCION_PROD':'NOMBRE_PRODUCTO',
                                                      'PRECIO_UNITARIO':'PRECIO'})

PRODUCTOS_closeup = PRODUCTOS_closeup.sort_values('NOMBRE_PRODUCTO')

PRODUCTOS_closeup.head()

,CODIGO_PRODUCTO,EAN,NOMBRE_PRODUCTO,LABORATORIO,PRECIO
19147,7709521995521,7709521995521,01NE DAY NARANJA X30 SOB PAG24 LLV30,NEW HEALTH,21143.200000000
33148,7709521995521,7709521995521,01NE DAY NARANJA X30 SOB PAG24 LLV30,NEW HEALTH,21143.200000000
46703,4104480706172,4104480706172,ABRILAR HEDERA HELIX 7MG/ML JARABE FRASCO X200...,MEGALABS,48899.550000000
48760,4104480706172,4104480706172,ABRILAR HEDERA HELIX 7MG/ML JARABE FRASCO X200...,MEGALABS,48899.550000000
825,4104480705151,4104480705151,ABRILAR HEDERA HELIX JARABE X100 ML,MEGALABS,34899.900000000


# CREACION ARCHIVOS EXCEL

In [ ]:
import datetime

# Get the current month in Spanish
meses = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre"
}

# Get the current month number
current_month = datetime.datetime.now().month
# Adjust for January
previous_month = current_month - 1 if current_month > 1 else 12
# Get the Spanish month name
spanish_month = meses[previous_month]

# Get the current year
current_year = datetime.datetime.now().year

### Archivo IQVIA

In [ ]:
# Create the file name with the current month and year
file_name = f'Data Farmu {spanish_month} {current_year} iqvia.xlsx'

with pd.ExcelWriter(file_name) as writer:  # doctest: +SKIP
  PRODUCTOS_iqvia.to_excel(writer, sheet_name='PRODUCTOS', index=False)
  CLIENTES_iqvia.to_excel(writer, sheet_name='CLIENTES', index=False)
  FACMES_iqvia.to_excel(writer, sheet_name='FACMES', index=False)

# Save the Excel file
writer.close()

/usr/local/lib/python3.11/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


#### Ajustando ancho de las columnas

In [ ]:
import openpyxl

# Load the Excel file
workbook = openpyxl.load_workbook(f'Data Farmu {spanish_month} {current_year} iqvia.xlsx')

# Get the list of sheet names in the workbook
sheet_names = workbook.sheetnames

# Iterate through each sheet
for sheet_name in sheet_names:
    # Select the current sheet
    worksheet = workbook[sheet_name]

    # Iterate through each column and autofit the width
    for column in worksheet.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except TypeError:
                pass
        adjusted_width = (max_length + 2) * 1.2  # Adjust the multiplication factor as per your preference
        worksheet.column_dimensions[column[0].column_letter].width = adjusted_width

# Save the modified workbook
# workbook.save(f'Data Farmu {spanish_month} {current_year} iqvia.xlsx')
workbook.close()

In [ ]:
help(workbook.close)

Help on method close in module openpyxl.workbook.workbook:

close() method of openpyxl.workbook.workbook.Workbook instance
    Close workbook file if open. Only affects read-only and write-only modes.



### Archivo CLOSEUP

In [ ]:
# Create the file name with the current month and year
file_name = f'Data Farmu {spanish_month} {current_year} closeup.xlsx'

with pd.ExcelWriter(file_name) as writer:  # doctest: +SKIP
  CLIENTES_closeup.to_excel(writer, sheet_name='CLIENTES', index=False)
  FACMES_closeup.to_excel(writer, sheet_name='FACMES', index=False)
  PRODUCTOS_closeup.to_excel(writer, sheet_name='PRODUCTOS', index=False)


# Save the Excel file
writer.close()

#### Ajustando ancho de las columnas

In [ ]:
import openpyxl

# Load the Excel file
workbook = openpyxl.load_workbook(f'Data Farmu {spanish_month} {current_year} closeup.xlsx')

# Get the list of sheet names in the workbook
sheet_names = workbook.sheetnames

# Iterate through each sheet
for sheet_name in sheet_names:
    # Select the current sheet
    worksheet = workbook[sheet_name]

    # Iterate through each column and autofit the width
    for column in worksheet.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except TypeError:
                pass
        adjusted_width = (max_length + 2) * 1.2  # Adjust the multiplication factor as per your preference
        worksheet.column_dimensions[column[0].column_letter].width = adjusted_width

# Save the modified workbook
# workbook.save(f'Data Farmu {spanish_month} {current_year} closeup.xlsx')
workbook.close()